In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import utils
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import matthews_corrcoef


In [2]:
### utilities here (move to new file eventually)
column_sums=pd.read_csv('column_sums.csv')
column_sums.set_index('Unnamed: 0', inplace=True)


for 

class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
class LogNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogNN, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes)


    def forward(self, x):
        out = self.fc1(x)
        return out


def isDivis(N, p):
    if N % p == 0:
        return 1
    else:
        return 0
    
def isDivisp(N, p):
    if N % p == 0:
        return 1/p
    else:
        return 0
    
def drawranddivs(N, prime):
    if prime > N:
        return 0
    else:
        print((df.iloc[:, :].sum(axis=0)/X.shape[0]).head(25))

    prob=column_sums.loc['isDivis_'+str(prime)].values[0]
    # Randomly sample either 0 or 1 based on the probability distribution
    sample = np.random.choice([0, 1], p=[1-prob, prob])
    return sample
    

def addRand():
    return random.random()
    
def preprocess(df, addcond=False , addprimeord=False, addrecipp=False, addRandom=0, drawrand=False, column_sums=False):
    X=df.drop(columns=['rank'])
    y=df['rank']

    column_names=df.columns.to_list()[:-2]
    column_names=[int(item) for item in column_names]
    
    if addprimeord:      
        new_columns={}
        for value in column_names:
            column_name = f'isDivis_{value}'
            new_columns[column_name] = X['conductor'].apply(lambda N: isDivis(int(N), value))
        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)

    if addRandom>0:    
        new_columns={}  
        for value in range(addRandom):
            column_name = f'randval_{value}'
            new_columns[column_name] = X['conductor'].apply(lambda N: addRand())
        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)
    
    if drawrand:
        new_columns={}
        for value in column_names:
            column_name = f'isDivis_{value}'
            prob=column_sums.loc['isDivis_'+str(value)].values[0]
            new_columns[column_name] = np.random.choice([0, 1], size=X.shape[0], p=[1-prob, prob])
        X = pd.concat([X, pd.DataFrame(new_columns, index=X.index)], axis=1)


    if addrecipp:
        new_columns={}
        for value in column_names:
            column_name = f'isDivisp_{value}'
            new_columns[column_name] = X['conductor'].apply(lambda N: isDivisp(int(N), value))
        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)
    
    if not addcond:
        X=X.drop(columns=['conductor'])
    else:
        X['conductor']=np.log(X['conductor'])

    return X, y

def gettens(X, y, random_state=42, test_size=0.2, batch_size=32):
    X_tensor = torch.tensor(X.values, dtype=torch.float32)
    y_tensor = torch.tensor(y.values, dtype=torch.long)

    #Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=test_size, random_state=random_state)

    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

def train(model, train_dataloader, input_size, num_classes, num_epochs=40, lr=0.001):
    # Instantiate the model, define the loss function and the optimizer
    model = model(input_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        for inputs, labels in train_dataloader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model


def evaluate_model_with_mcc(model, test_dataloader):
    model.eval()  # Set the model to evaluation mode

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            # Store labels and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Compute MCC
    mcc = matthews_corrcoef(all_labels, all_predictions)
    return mcc


In [3]:
path = r"../data_files/ecq_B_1000_all_one_per_iso_262143_524287_sample_80k.parquet"
df = utils.load_data(path)

Loaded the big dataset with 1000 a_p's and 80000 curves..


In [4]:
df1=df[(df['conductor']>= 400000)]
del df

In [5]:
#df1
primes=df1[df1['conductor', 'rank']]

,2,3,5,7,11,13,17,19,23,29,...,7867,7873,7877,7879,7883,7901,7907,7919,conductor,rank
index,,,,,,,,,,,,,,,,,,,,,
2100582,0.0000,-0.2887,-0.2236,0.0000,0.0000,0.1387,0.4851,0.2294,0.4170,-0.5571,...,-0.1917,-0.9129,0.6648,0.4506,0.1126,0.5513,0.0112,-0.1124,485160,0
2077622,0.0000,0.0000,0.8944,-0.1890,0.4523,0.0000,0.1213,0.0000,-0.9383,-0.3714,...,-0.3495,-0.4283,-0.1690,-0.0225,-0.7771,-0.7425,-0.7647,-0.1011,479791,1
2033639,0.3536,0.0000,-0.4472,-0.3780,-0.6030,0.8321,0.2425,0.0000,-0.4170,-0.1857,...,-0.4510,0.7776,0.2366,0.0225,0.6533,0.0000,-0.6523,0.4944,469386,1
1848068,-0.7071,0.0000,0.0000,0.7559,-0.3015,-0.5547,0.2425,-0.1147,0.8341,0.9285,...,0.1466,-0.4846,0.9183,-0.5295,0.2196,-0.5906,-0.3318,0.0225,425277,1
2158368,0.0000,-0.2887,0.0000,0.0000,-0.3015,0.1387,0.7276,0.5735,0.6255,0.0000,...,-0.4453,-0.1409,0.1014,-0.2929,0.0788,0.7425,0.1125,-0.2697,498624,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1846446,0.3536,-0.2887,0.2236,0.1890,0.1508,0.5547,0.9701,0.5735,-0.1043,0.4642,...,0.7216,0.5297,0.1859,-0.1972,-0.0338,-0.8325,-0.4892,0.3371,424886,1
2126163,-0.3536,-0.5774,0.0000,-0.1890,-0.9045,0.0000,-0.2425,0.4588,0.1043,-0.1857,...,-0.2931,0.5748,0.7098,0.4281,0.4167,-0.2025,-0.7872,0.1011,491050,1
1927411,0.7071,0.5774,-0.6708,0.7559,-0.1508,0.8321,0.3638,-0.2294,-0.8341,-0.3714,...,-0.3495,0.5297,-0.1746,0.1465,-0.1295,-0.0225,0.7422,0.7023,444103,1


In [6]:
### set up parameters
addcond=False
addprimeord=False
addrecipp=False
model=LogNN
addRandom=0
drawrand=True

X, y = preprocess(df1, addcond=addcond, addprimeord=addprimeord, addrecipp=addrecipp, addRandom=addRandom, drawrand=drawrand, column_sums=column_sums)
input_size = X.shape[1]
print(X.shape)

#print((X.iloc[:, -1000:].sum(axis=0)/X.shape[0]).head(25))
#print(column_sums.head(25))


(33596, 2000)


In [12]:
mccs=[]
for i in range(100):     
    num_classes = len(y.unique())
    train_dataloader, test_dataloader = gettens(X, y, random_state=i)
    model = train(LogNN, train_dataloader, input_size, num_classes)
    mcc = evaluate_model_with_mcc(model, test_dataloader)
    mccs.append(mcc)
    print(np.mean(mccs), np.std(mccs))

print(mccs)

0.7671408751187908 0.0
0.7680502733973802 0.0009093982785893795
0.7696759404652732 0.002415972609667774
0.7710975045192401 0.0032311338283317957
0.772975971737649 0.004739908967039311
0.774137664247217 0.005046774046898574
0.7749518837619858 0.0050802645213520806
0.7754020853231138 0.004899155210394993
0.7766663411738666 0.005841370271586962
0.7767633296652137 0.005549243812618625
0.7769058084826476 0.005310145455932519
0.7762619252528292 0.005514370569969992
0.7762988681268228 0.005299581249363623
0.7765659695650947 0.005196817411354374
0.7762889733218313 0.00512646343230636
0.776579407523699 0.005089535194820776
0.776193820500873 0.00517285980626623
0.7763946716364343 0.0050948696224112945
0.7769065248337865 0.005413629876512119
0.7772784269227062 0.0055199575296359955
0.7773024255759854 0.0053879959850383335
0.77776006306817 0.005666480925484916
0.7780573433089468 0.005714649967457547
0.7779014946595971 0.0056440365881212365
0.7779241958823523 0.005531122068810199
0.7780257700522711

KeyboardInterrupt: 

In [24]:
X.shape

(67192, 3000)

In [26]:
print(mccs)

[0.7878364794812988, 0.7947378003732951, 0.7870901746188633, 0.7548575962313033, 0.8072042865382699, 0.8124701438101052, 0.8384511190173045, 0.8148520670192686, 0.7886854667227274, 0.7817801991886838, 0.8008939539470611, 0.7330713068343708, 0.7986100885140449, 0.7310944216862689, 0.8104509018994664, 0.8137164667743187, 0.7976931440999939, 0.819064176602586, 0.8201633232856547, 0.8262146455439293, 0.7982831868154857, 0.8097453944547962, 0.826580843991484, 0.7972724290437134, 0.8321218220801132, 0.7783092264174348, 0.8208887872837057, 0.8089732949991915, 0.7146663701265891, 0.8001099896021562, 0.8279849391183957, 0.774834979048887, 0.8310578323078973, 0.8299981397495454, 0.7981928617193585, 0.8463222749487738, 0.80059923681831, 0.8399882093197663, 0.815470912243347, 0.8304127256345094, 0.8026748105122686, 0.8137152674699702, 0.792501962963442, 0.837691283936441, 0.8099557903394708, 0.8025198997822448, 0.8272151159441301, 0.8273612211913431, 0.7858503025953711, 0.7754507387995626, 0.80732

In [9]:
mccs1=[0.8089224248231514, 0.822285431792844, 0.8098035743943567, 0.8127132684016201, 0.8218982166604264, 0.8183435230707452, 0.8394283483721894, 0.8093546207918101, 0.8154759859327243, 0.8013678403093563, 0.8024133193486871, 0.8185288626709916, 0.8271026924313526, 0.8086639519505081, 0.8217840729783134, 0.827650795048157, 0.8117372646727923, 0.8159249605614677, 0.8212189867954669, 0.819368751877494, 0.8044544071975132, 0.8177424207646712, 0.8154435003816732, 0.8005389321113462, 0.8069060072928397, 0.8185593176463711, 0.8297172117898084, 0.8241412049051807, 0.8230412347066447, 0.7981097570681481, 0.8111273195782855, 0.8126630086164955, 0.8186220588443727, 0.8073353861061883, 0.8395495639218606, 0.8028364793736076, 0.8099657324514615, 0.8138963504507272, 0.822200925011414, 0.8162592944845937, 0.8166086732364134, 0.8166372533236044, 0.8228093815543885, 0.79941809309625, 0.8244213116421142, 0.8341175562062111, 0.7847197195924368, 0.8119862365993674, 0.8125165986726668, 0.8180092945971117, 0.8188139507297414, 0.8159024536953247, 0.8223869581955655, 0.832732692562796, 0.827745232537455, 0.8114396495269212, 0.7956483867586953, 0.8136117731067739, 0.8220172156485327, 0.8036178139422522, 0.8128722367590113, 0.8141936949632306, 0.8108647528262969, 0.8095355775152654, 0.8311743829075292, 0.8073962011830497, 0.806881331461867, 0.810021184144381, 0.8075397817830479, 0.8076294419623099, 0.8084433091012142, 0.8242191434396937, 0.817235052584041, 0.8372621165041195, 0.804619740189814, 0.7926585355050128, 0.8200771926271385, 0.8026831028063546, 0.8199200149645095, 0.8277589634152214, 0.8252389360678751, 0.8260464311040656, 0.812970423257097, 0.8121225565088973, 0.8119343903269546, 0.8011482358217997, 0.8217483300733547, 0.8132278880283641, 0.7986649768778303, 0.828670922824582, 0.8103438259968345, 0.8128068018201878, 0.8098588633107949, 0.7986370897808562, 0.8102215747327906]
print(len(mccs1))

95


In [10]:
mcc40fftl=mccs+mccs1

In [11]:
print(mcc40fftl)

[0.8213717802912285, 0.7920843792784381, 0.8158457863127198, 0.8078571436791488, 0.8241466970168229, 0.8089224248231514, 0.822285431792844, 0.8098035743943567, 0.8127132684016201, 0.8218982166604264, 0.8183435230707452, 0.8394283483721894, 0.8093546207918101, 0.8154759859327243, 0.8013678403093563, 0.8024133193486871, 0.8185288626709916, 0.8271026924313526, 0.8086639519505081, 0.8217840729783134, 0.827650795048157, 0.8117372646727923, 0.8159249605614677, 0.8212189867954669, 0.819368751877494, 0.8044544071975132, 0.8177424207646712, 0.8154435003816732, 0.8005389321113462, 0.8069060072928397, 0.8185593176463711, 0.8297172117898084, 0.8241412049051807, 0.8230412347066447, 0.7981097570681481, 0.8111273195782855, 0.8126630086164955, 0.8186220588443727, 0.8073353861061883, 0.8395495639218606, 0.8028364793736076, 0.8099657324514615, 0.8138963504507272, 0.822200925011414, 0.8162592944845937, 0.8166086732364134, 0.8166372533236044, 0.8228093815543885, 0.79941809309625, 0.8244213116421142, 0.834

In [27]:
mcc40fffl=[0.7946355874441422, 0.7970099225979029, 0.7881631873744196, 0.804410492104615, 0.804911941723888, 0.7976156277119161, 0.8124026191058324, 0.7998299711157842, 0.8021585228692502, 0.7967752251454263, 0.8197014728728946, 0.8076668170099922, 0.8004400658461456, 0.7631617924200491, 0.8095123924723484, 0.7899495919370301, 0.808058383813526, 0.8095995157042086, 0.8022690927377584, 0.7903370932878641, 0.8202731611519917, 0.8076076414570362, 0.7918396695673627, 0.793169315011796, 0.7980568153736584, 0.810229434793833, 0.8159783592118043, 0.8022529733346683, 0.8066074670446297, 0.8130107534735843, 0.8043436457530931, 0.7942675704174891, 0.8140830232545405, 0.7847908241747936, 0.805804929950695, 0.7985739979817161, 0.8079903106394124, 0.7616148155938701, 0.7921837429931943, 0.826329544830494, 0.7829747678516574, 0.8258673736183162, 0.7927358358186557, 0.8110504755630863, 0.807836450303094, 0.8078872217049669, 0.7990498983797394, 0.819819862908979, 0.7987124466773032, 0.8027358440550357, 0.8035940720728303, 0.7939298622965262, 0.8026708116400691, 0.7988971427874882, 0.8058731745446308, 0.797776731217904, 0.8026518706263653, 0.8212437843953339, 0.8018585557432577, 0.8072187383696002, 0.750257009737517, 0.8022397648865641, 0.8128010847128644, 0.798416162695093, 0.7974416337493268, 0.7942628515633684, 0.8116748454030649, 0.7994293865556448, 0.8054489130189293, 0.8141048485385967, 0.7960998453130066, 0.816359541279036, 0.8101305773230666, 0.809309292429526, 0.7966973615108798, 0.8196665567353372, 0.8099322458845509, 0.7975278288372175, 0.8180412849972871, 0.8087711678268917, 0.8047103214420545, 0.7909276936739932, 0.8062712581680858, 0.7991017521470306, 0.7867505677208094, 0.8045609932371881, 0.8008043086886596, 0.8107986008283956, 0.8176994161908768, 0.7972046587378694, 0.7877873293617839, 0.8004209118248294, 0.8161817802565491, 0.783796647438462, 0.7909655938281323, 0.8055753187075213, 0.8076006511684184, 0.8013195786506547, 0.8075463872814821, 0.7944134802873589]
mcc40fffladd1000=[0.7878364794812988, 0.7947378003732951, 0.7870901746188633, 0.7548575962313033, 0.8072042865382699, 0.8124701438101052, 0.8384511190173045, 0.8148520670192686, 0.7886854667227274, 0.7817801991886838, 0.8008939539470611, 0.7330713068343708, 0.7986100885140449, 0.7310944216862689, 0.8104509018994664, 0.8137164667743187, 0.7976931440999939, 0.819064176602586, 0.8201633232856547, 0.8262146455439293, 0.7982831868154857, 0.8097453944547962, 0.826580843991484, 0.7972724290437134, 0.8321218220801132, 0.7783092264174348, 0.8208887872837057, 0.8089732949991915, 0.7146663701265891, 0.8001099896021562, 0.8279849391183957, 0.774834979048887, 0.8310578323078973, 0.8299981397495454, 0.7981928617193585, 0.8463222749487738, 0.80059923681831, 0.8399882093197663, 0.815470912243347, 0.8304127256345094, 0.8026748105122686, 0.8137152674699702, 0.792501962963442, 0.837691283936441, 0.8099557903394708, 0.8025198997822448, 0.8272151159441301, 0.8273612211913431, 0.7858503025953711, 0.7754507387995626, 0.8073235143928106, 0.7755822223848868, 0.8230056117921754, 0.8049266399704181, 0.7616494189978796, 0.8059540283630214, 0.8082395239439744, 0.8110753876070609, 0.7763251378571073, 0.8362330943991111, 0.8368223180236743, 0.8163353262581223, 0.8037332406353663, 0.7845014725705395, 0.8242311661620708, 0.7999480820222972, 0.8023115238558527, 0.7819269062787886, 0.7945248804342533, 0.7736894694822254, 0.8176038541440769, 0.7932181394263047, 0.7913188418499554, 0.8584781913026474, 0.7972073911343446, 0.8180740738100383, 0.7311201247238417, 0.769318099629834, 0.7938773113580216, 0.8252049848075594, 0.8067832005149683, 0.8187051467669165, 0.7934163405541438, 0.8093669740574116, 0.7560360418977066, 0.8302074908829346, 0.8203124866728799, 0.7743030781253245, 0.8081189215207286, 0.738841809151216, 0.7904571600502336, 0.7717434862669968, 0.8093938490211999, 0.7757340042881686, 0.8184597510112789, 0.8146204883309394, 0.79760055807609, 0.7830846146586198, 0.7975078474974835, 0.7919185419024083]
mcc40tffl=[0.784086977454015, 0.7696769441397194, 0.8079583442845667, 0.8169989153213546, 0.8211188800077663, 0.8212997498349408, 0.8161600947974037, 0.7812991002677263, 0.7561523074798929, 0.8023603208915735, 0.7892907725319215, 0.7788968315882707, 0.8101393663078459, 0.8227274925974907, 0.8117641165505983, 0.8069150669130699, 0.7955015241878293, 0.8006385628707472, 0.7970086084153015, 0.8179376305052596, 0.8229342483295082, 0.7940691520897546, 0.7997064951920819, 0.822680891369416, 0.7972081449869872, 0.7852718268587165, 0.8065716619621065, 0.8221893655400749, 0.8223075010436107, 0.8175129724599559, 0.7955750804639341, 0.8130775137069646, 0.8141003949140813, 0.8189841003992118, 0.8262525566942827, 0.8134873879308988, 0.7769318244852711, 0.7995149113156717, 0.7991693506280572, 0.8234429581488419, 0.7895045703303422, 0.8091495905369996, 0.7808501514659061, 0.8212466842255012, 0.7801295330693683, 0.8189904102210497, 0.8112129573851284, 0.8099073467995318, 0.8109493112713211, 0.8259606082881041, 0.8036296896349805, 0.7852283131120963, 0.8075713118937803, 0.7874661487183756, 0.8207475648958481, 0.796280949604311, 0.8031573997413399, 0.7821399726502537, 0.7857833139485874, 0.7882748976838219, 0.7805264366286327, 0.7710311822290242, 0.8065104279547175, 0.8200356427760347, 0.8155010985856807, 0.8074051385833769, 0.794398129503646, 0.7853733791005479, 0.7792768827320203, 0.8271403364338147, 0.8244883167382762, 0.8194202959863981, 0.8170149035368923, 0.8039893151291522, 0.8004598692639578, 0.8227780262154775, 0.787207950361665, 0.8090808373642739, 0.8193959428287432, 0.7642963012587602, 0.8132172202753052, 0.846354577010815, 0.817802856015131, 0.7838559914236818, 0.7999530601887999, 0.8117452774806974, 0.7964425705540847, 0.7779739816164358, 0.8106882820526119, 0.7845844799634535, 0.825784160361629, 0.8123831571955669, 0.7924893878829192, 0.7849065890366448, 0.8209667654105084, 0.796120532964439, 0.8282573776262128, 0.8331122957642477, 0.8000901722709025, 0.8023197676048897]
mcc40ftfl=[0.821494935874429, 0.81874836911681, 0.795704031333243, 0.8173565474029877, 0.8251979158519511, 0.7974135215968565, 0.8376377595416673, 0.8300988145404213, 0.814942656612831, 0.8075388926629686, 0.8286474805596089, 0.8059681543388378, 0.8237349796309884, 0.8295676233654988, 0.8161036206495402, 0.8191770876598298, 0.8169660383268721, 0.796547401541924, 0.8037881730236647, 0.815574384569892, 0.8264012504626624, 0.8151551487944987, 0.8123910720833625, 0.826237925908889, 0.8043734577019406, 0.8241765257088872, 0.8270417975661798, 0.8204886205516274, 0.8158255276992759, 0.810214743692916, 0.815256056046846, 0.8034010092930917, 0.7748509751354822, 0.8149469571778433, 0.8190952985332562, 0.820534044691442, 0.8343018912279075, 0.8136398806130622, 0.8284126788065355, 0.8135341109195302, 0.8159088722445385, 0.8372511743789988, 0.8270051125136321, 0.8160610876372972, 0.8302126415863694, 0.8179004194478074, 0.8018236609734914, 0.7994195662252114, 0.8217527342238624, 0.8076571563630424, 0.8123982164103849, 0.8112785642721275, 0.809489457659292, 0.8101505585214482, 0.8266567501142504, 0.8290862139063282, 0.8231271794160109, 0.8049247292731182, 0.8054835066909435, 0.823586215517658, 0.8073716475678967, 0.8222656524240491, 0.8114188018244842, 0.8118996218501238, 0.8164654488833623, 0.8139698245666689, 0.796826674017777, 0.7980434355204943, 0.7914890200149519, 0.8071644639251823, 0.8184933020639823, 0.8051616677898317, 0.8188876699155788, 0.8173003405320916, 0.8129599473123775, 0.8272737707427045, 0.8033797487927021, 0.8184477252369018, 0.7855615689572056, 0.8443398355825567, 0.8105130700607642, 0.820911526081122, 0.8024626971404452, 0.7998839761222062, 0.819139658845979, 0.8054283593799418, 0.8347602437033024, 0.80518664276337, 0.8137090182021058, 0.821417336629458, 0.8075114493833344, 0.8148801156002343, 0.7988562455509547, 0.8214367864452274, 0.8092501223318533, 0.8132358037471529, 0.8140035271317838, 0.8179813405165394, 0.8143924962319881, 0.821153493299781]
mcc40fftl=[0.8213717802912285, 0.7920843792784381, 0.8158457863127198, 0.8078571436791488, 0.8241466970168229, 0.8089224248231514, 0.822285431792844, 0.8098035743943567, 0.8127132684016201, 0.8218982166604264, 0.8183435230707452, 0.8394283483721894, 0.8093546207918101, 0.8154759859327243, 0.8013678403093563, 0.8024133193486871, 0.8185288626709916, 0.8271026924313526, 0.8086639519505081, 0.8217840729783134, 0.827650795048157, 0.8117372646727923, 0.8159249605614677, 0.8212189867954669, 0.819368751877494, 0.8044544071975132, 0.8177424207646712, 0.8154435003816732, 0.8005389321113462, 0.8069060072928397, 0.8185593176463711, 0.8297172117898084, 0.8241412049051807, 0.8230412347066447, 0.7981097570681481, 0.8111273195782855, 0.8126630086164955, 0.8186220588443727, 0.8073353861061883, 0.8395495639218606, 0.8028364793736076, 0.8099657324514615, 0.8138963504507272, 0.822200925011414, 0.8162592944845937, 0.8166086732364134, 0.8166372533236044, 0.8228093815543885, 0.79941809309625, 0.8244213116421142, 0.8341175562062111, 0.7847197195924368, 0.8119862365993674, 0.8125165986726668, 0.8180092945971117, 0.8188139507297414, 0.8159024536953247, 0.8223869581955655, 0.832732692562796, 0.827745232537455, 0.8114396495269212, 0.7956483867586953, 0.8136117731067739, 0.8220172156485327, 0.8036178139422522, 0.8128722367590113, 0.8141936949632306, 0.8108647528262969, 0.8095355775152654, 0.8311743829075292, 0.8073962011830497, 0.806881331461867, 0.810021184144381, 0.8075397817830479, 0.8076294419623099, 0.8084433091012142, 0.8242191434396937, 0.817235052584041, 0.8372621165041195, 0.804619740189814, 0.7926585355050128, 0.8200771926271385, 0.8026831028063546, 0.8199200149645095, 0.8277589634152214, 0.8252389360678751, 0.8260464311040656, 0.812970423257097, 0.8121225565088973, 0.8119343903269546, 0.8011482358217997, 0.8217483300733547, 0.8132278880283641, 0.7986649768778303, 0.828670922824582, 0.8103438259968345, 0.8128068018201878, 0.8098588633107949, 0.7986370897808562, 0.8102215747327906]




print(np.mean(mcc40fffl), np.std(mcc40fffl))
print(np.mean(mcc40tffl), np.std(mcc40tffl))
print(np.mean(mcc40ftfl), np.std(mcc40ftfl))
print(np.mean(mcc40fftl), np.std(mcc40fftl))

0.802150516845174 0.012070663904107529
0.8039147958685342 0.01710229271791245
0.8146849525488032 0.011346647696314273
0.8146019004056604 0.010281492111229498


In [30]:
import numpy as np
from scipy import stats

# Assume list_a and list_b are your two lists of 100 values each
list_a = mcc40ftfl
list_b = mcc40fffladd1000

# Convert lists to numpy arrays for ease of calculation
array_a = np.array(list_a)
array_b = np.array(list_b)

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(array_a, array_b)

print(f'T-statistic: {t_stat}')
print(f'P-value: {p_value}')

# Determine if the results are significant
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two lists.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two lists.")


T-statistic: 4.779464230049409
P-value: 3.420629948087639e-06
Reject the null hypothesis: There is a significant difference between the two lists.
